<a href="https://colab.research.google.com/github/mpodlasin/mlsg-kaggle/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic

## Ładowanie danych

`kaggle` jest to skrypt pozwalający w prosty sposób pobierać dane z portalu Kaggle. Najpierw musimy go zanistalować:

(zwróć uwagę, że komenda zaczyna się od znaku `!`, oznacza to, jest to komenda języka `bash` a nie Python)

In [ ]:
!pip install kaggle

Aby skrypt `kaggle` działał poprawnie, musimy założyć sobie konto na https://kaggle.com i dodać się do konkursu z którego będziemy ściągać dane (w tym przypadku https://www.kaggle.com/c/titanic).

Po zrobieniu tego, z portalu kaggle musimy ściągnąć specjalny klucz (w postaci pliku JSON), który pozwala portalowi kaggle potwierdzić, że my to faktycznie my i że mamy dostęp do konkursu. W tym celu idziemy do: https://www.kaggle.com/twoj_login/account i klikamy przycisk "Create New API Token". Spowoduje to ściągnięcie pliku `kaggle.json`.

Następnie używamy następującego kodu aby zrobić uppload tego pliku do naszego notebooka:

In [ ]:
from google.colab import files

files.upload()

Skrypt `kaggle` wymaga, żeby plik `kaggle.json` znajdował się w folderze `.kaggle` w roocie, naszego drzewa katalogów.

Używamy więc komend bashowych, żeby stworzyć taki katalog i umieścić tam plik:

In [ ]:
# tworzymy katalog
!mkdir -p ~/.kaggle

# przenosimy plik do katalogu
!cp -rf kaggle.json ~/.kaggle/

# nadajemy uprawnienia do odczytu pliku
!chmod 600 ~/.kaggle/kaggle.json

Teraz możemy już ściągać datasety za pomocą skryptu `kaggle`.

Sciągamy więc dataset Titanic:

In [ ]:
!kaggle competitions download -c titanic

## Eksploracja danych

### Dataset treningowy

In [ ]:
import pandas as pd

In [ ]:
train_titanic = pd.read_csv('/content/train.csv')

In [ ]:
print(train_titanic)

### Dataset testowy

In [ ]:
test_titanic = pd.read_csv('/content/test.csv')

In [ ]:
print(test_titanic)